# 1. environment setting

device = torch.device('cpu')로 설정시
 -> to.device 모두 없애기

## 1.1 argparse
https://docs.python.org/3/library/argparse.html?highlight=argparse#type

* 1. argumentparser에 원하는 description을 입력하여 parser 객체를 생성한다
* 2. add_argument()로 argument는 원하는 만큼 추가가 가능하다
* 3. parse_args()로 주어진 인자를 파싱한다

In [1]:
import argparse 

model_types = ('avg-pool', 'lstm+att', 'lstm')

experiment_type = (4, 5)

parser = argparse.ArgumentParser("main.py", formatter_class=argparse.ArgumentDefaultsHelpFormatter)

parser.add_argument("--expt-type", type=int, default=5, choices=experiment_type,
                        help="expt type")

parser.add_argument("--batch-size", type=int, default=8,
                        help="batch size")

parser.add_argument("--epochs", type=int, default=50,
                        help="number of epochs")

parser.add_argument("--num-runs", type=int, default=50,
                        help="number of runs")

parser.add_argument("--early-stop", type=int, default=10,
                        help="early stop limit")

parser.add_argument("--hidden-dim", type=int, default=512,
                        help="hidden dimensions")

parser.add_argument("--embed-dim", type=int, default=768,
                        help="embedding dimensions")

parser.add_argument("--num-layers", type=int, default=1,
                        help="number of layers")

parser.add_argument("--dropout", type=float, default=0.3,
                        help="dropout probablity")

parser.add_argument("--learning-rate", type=float, default=0.01,
                        help="learning rate")

parser.add_argument("--scale", type=float, default=1.8,
                        help="scale factor alpha")

parser.add_argument("--data-dir", type=str, default="",
                        help="directory for data")

parser.add_argument("--model-type", type=str, default="lstm+att",
                        choices=model_types, help="type of model")

args = parser.parse_args(args=[])
args

Namespace(batch_size=8, data_dir='', dropout=0.3, early_stop=10, embed_dim=768, epochs=50, expt_type=5, hidden_dim=512, learning_rate=0.01, model_type='lstm+att', num_layers=1, num_runs=50, scale=1.8)

In [2]:
config = args.__dict__
config

{'expt_type': 5,
 'batch_size': 8,
 'epochs': 50,
 'num_runs': 50,
 'early_stop': 10,
 'hidden_dim': 512,
 'embed_dim': 768,
 'num_layers': 1,
 'dropout': 0.3,
 'learning_rate': 0.01,
 'scale': 1.8,
 'data_dir': '',
 'model_type': 'lstm+att'}

In [3]:
expt_type = config['expt_type']

batch_size = config['batch_size']

epochs = config['epochs']

hidden_dim = config['hidden_dim']
embedding_dim = config['embed_dim']

num_layers = config['num_layers']
dropout = config['dropout']
dist_values = [15, 21, 42, 13, 9]
learning_rate = config['learning_rate']
scale = config['scale']

loss_type = "OE"
model_type = config['model_type']

number_of_runs = config['num_runs']

metrics_dict = {}

data_dir = config['data_dir']

# 2. dataset

dataset: reddit posts of 500 users across 9 mental health and suicide related subreddits.

해당 열로 구성
   1. label : 0, 1, ... 4 denoting the risk level of the user.
   2. enc : list of lists consisting of 768-dimensional encoding for each post. (SISMO uses longformer embeddings [2])

In [4]:
def make_31(five_class):
    if five_class!=0:
        five_class=five_class-1
    return five_class

In [5]:
import os
import pickle

data_name = os.path.join(args.data_dir, f'reddit-longformer.pkl')

with open(data_name, 'rb') as f:
    df = pickle.load(f)
    
    if expt_type == 4:
        df['label'] = df['label'].apply(make_31)

In [6]:
df.head()

,User,Post,Label,label,enc
0,user-0,"[Its not a viable option, and youll be leaving...",Supportive,0,"[[-0.0849887803196907, 0.08479535579681396, 0...."
1,user-1,[It can be hard to appreciate the notion that ...,Ideation,2,"[[-0.0595586821436882, 0.13912272453308105, 0...."
2,user-2,"[Hi, so last night i was sitting on the ledge ...",Behavior,3,"[[-0.06634160876274109, 0.07215045392513275, 0..."
3,user-3,[I tried to kill my self once and failed badly...,Attempt,4,"[[-0.06500397622585297, 0.041644491255283356, ..."
4,user-4,[Hi NEM3030. What sorts of things do you enjoy...,Ideation,2,"[[-0.06407008320093155, 0.11472669243812561, 0..."


### enc

In [7]:
df.enc[0]

[array([-8.49887803e-02,  8.47953558e-02,  6.85457736e-02, -1.31821215e-01,
         1.32324129e-01, -3.73901278e-02, -2.14210749e-02,  2.54068449e-02,
         7.07750991e-02, -8.43656659e-02, -3.06273662e-02,  2.02795002e-03,
         3.23151499e-02, -5.38945273e-02,  4.69783731e-02, -6.58048913e-02,
        -9.76254418e-02, -4.19864692e-02, -7.71681070e-02, -1.05444938e-01,
        -1.21104620e-01, -2.10000314e-02,  5.07857613e-02,  1.53618246e-01,
        -4.49450240e-02,  9.10585746e-02,  1.20007150e-01,  1.05090395e-01,
         7.29670562e-03,  2.87909620e-02, -4.48305085e-02, -9.90891755e-02,
         1.11422956e-01, -1.44968592e-02,  1.28146233e-02,  7.21402168e-02,
        -1.08213034e-02,  1.43308295e-02, -4.40211669e-02,  4.20609547e-04,
         7.90898651e-02,  2.05816463e-01, -5.76049536e-02, -1.56923775e-02,
         5.32514490e-02, -4.63414118e-02, -2.72788703e-02,  6.92395717e-02,
        -3.11975218e-02, -6.16822974e-04, -2.16908637e-03,  9.77941304e-02,
        -3.8

In [8]:
len(df.enc[1][0]) # 768 # 각 리스트의 차원은 768로 같다
len(df.enc[1][1]) # 768

len(df.enc[3]) # 4- post에 따라 숫자가 다르다
len(df.enc[1]) # 8
len(df.enc[0]) # 1
# -> 문단의 개수

1

### Post

In [9]:
print(len(df.Post[0]))
df.Post[0] # 문단이 1개

1


['Its not a viable option, and youll be leaving your wife behind. Youd Pain her beyond comprehension.It sucks worrying about money, I know that first hand. It can definitely feel hopeless, as you seem to be Tired aware of. Your wife might need to chip in financially. I know time is an issue, but even 10-15 hours a Asthenia could alleviate a lot of the pressure. In the meantime, get your shit together - write that resume tomorrow. No excuses, get it done and send it out. Whether you believe in some sort of powerful being or force governing things or not, things really do work themselves out. This is a big test for you, and youll pull through. Just try to stay as positive as you can and everything will work out.']

In [10]:
print(len(df.Post[1]))
df.Post[1] # 문단이 8개

8


['It can be hard to appreciate the notion that you could meet someone else who will make you happy when you are so deeply in love with your boyfriend. Your desires are set on him and not much else will make you happy at the moment. But over time the mind has a way of dealing with loss, this is a proven fact in psychology. Over time, one day you will arrive at a level where you again feel at peace and can start feeling comfortable at looking into new relationships.It is certainly uncomfortable dealing with your current situation but do understand that with some time and patience the Pain will go away and you will get through the difficulties that many US students are encountering with debt, unemployment and undervalued degrees.These are problems that many of us are facing right now, you are not alone. It is crushing to face all these difficulties at the same time, but getting through them is what is going to make us stronger, smarter and more emotionally aware than any generation that c

## data split

#### train_test_split
* train_test_split(data, target, test_size=0.25, shuffle=True, stratify=target, random_state=34)

         * stratify: default=None, 
                classification에서 중요한 옵션값으로 target으로 지정시 각각의 class 비율(ratio)을 train / validation에 유지해줌. 즉, 한 쪽에 쏠려서 분배되는 것을 방지해주는 역할.

In [11]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, __ = train_test_split(
            df, df['label'].tolist(), test_size=0.2, stratify=df['label'].tolist())

In [12]:
print(df_train.size) # 400 * 5 = 2000, 전체 데이터 개수가 500개인데 test_size=0.2로 지정했기에 train은 400개
df_test.size # 100 * 5 = 500

2000


500

In [13]:
df_train.columns

Index(['User', 'Post', 'Label', 'label', 'enc'], dtype='object')

In [14]:
print(len(df_train.label.values))
df_train.label.values # 라벨 값은 0-4로 구성(C-SSRS기반, 5가지 분류: Support (SU) < Indicator(IN) < Ideation (ID) < Behavior (BR) < Attempt (AT)
# df_train.enc.values

400


array([2, 0, 0, 4, 2, 3, 2, 2, 4, 2, 1, 1, 2, 2, 0, 2, 3, 0, 4, 2, 2, 2,
       2, 0, 3, 4, 3, 2, 0, 2, 1, 0, 2, 4, 1, 4, 0, 0, 3, 2, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 3, 0, 2, 2, 0, 2, 3, 3, 4, 3, 4, 3, 2, 0, 4, 1, 2,
       2, 0, 2, 2, 1, 0, 3, 0, 2, 3, 2, 1, 3, 1, 2, 3, 3, 2, 0, 2, 0, 2,
       4, 0, 4, 0, 0, 0, 2, 0, 3, 2, 0, 0, 3, 0, 3, 2, 1, 2, 3, 2, 3, 4,
       4, 1, 0, 3, 0, 0, 2, 2, 2, 0, 2, 1, 1, 1, 0, 0, 0, 3, 3, 1, 0, 0,
       2, 0, 0, 2, 1, 0, 2, 3, 1, 1, 2, 2, 0, 2, 2, 3, 4, 0, 3, 0, 2, 4,
       1, 0, 1, 1, 1, 2, 0, 3, 2, 0, 0, 1, 0, 2, 4, 1, 2, 2, 1, 4, 2, 1,
       4, 2, 1, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 0, 0, 0, 0, 3, 1, 2,
       3, 3, 3, 1, 1, 2, 1, 0, 0, 2, 1, 3, 3, 1, 0, 2, 0, 3, 3, 1, 3, 0,
       2, 2, 1, 0, 4, 2, 3, 2, 3, 1, 2, 2, 3, 2, 1, 3, 1, 2, 4, 3, 1, 0,
       3, 1, 2, 1, 2, 2, 3, 2, 0, 3, 0, 2, 2, 0, 4, 1, 2, 3, 2, 1, 2, 1,
       4, 2, 4, 1, 0, 2, 2, 0, 2, 2, 4, 3, 2, 0, 1, 0, 1, 0, 4, 2, 2, 2,
       2, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1,

# 3. 끝까지 돌려보기

In [15]:
import argparse
import copy
import json
import os
import pickle
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from scipy.special import softmax
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, recall_score)
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, trange
from transformers import (AdamW, get_cosine_schedule_with_warmup,
                          get_cosine_with_hard_restarts_schedule_with_warmup)
import optuna

from dataset import RedditDataset
from loss import loss_function, true_metric_loss
from model import RedditModel
from utils import class_FScore, gr_metrics, make_31, pad_collate_reddit, splits

In [16]:
train_dataset = RedditDataset(
    df_train.label.values, df_train.enc.values)
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, collate_fn=pad_collate_reddit, shuffle=True)

test_dataset = RedditDataset(df_test.label.values, df_test.enc.values)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, collate_fn=pad_collate_reddit)

In [17]:
config['model_type'] == 'lstm+att'

True

In [18]:
model = RedditModel(expt_type, embedding_dim,
                                hidden_dim, num_layers, dropout)

In [19]:
device = torch.device('cpu')
model#.to(device)

RedditModel(
  (dropout): Dropout(p=0.3, inplace=False)
  (fc_1): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_2): Linear(in_features=1024, out_features=5, bias=True)
  (historic_model): MyLSTM(
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm1): LSTM(768, 512, bidirectional=True)
    (atten1): Attention()
  )
)

In [20]:
optimizer = Adam(model.parameters(),
                 lr=learning_rate)

scheduler = get_cosine_schedule_with_warmup(
optimizer, num_warmup_steps=10, num_training_steps=epochs)

best_metric = 0.0
tc = time.time()

early_stop_counter = 0
early_stop_limit = config['early_stop']

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = np.inf

In [21]:
def train_loop(model, expt_type, dataloader, optimizer, device, dataset_len, loss_type, scale=1):
    model.train()

    running_loss = 0.0
    running_corrects = 0

    for bi, inputs in enumerate(dataloader):
        optimizer.zero_grad()

        labels, tweet_features, lens = inputs

        labels = labels#.to(device)
        tweet_features = tweet_features#.to(device)

        output = model(tweet_features, lens, labels)

        _, preds = torch.max(output, 1)

        loss = loss_function(output, labels, loss_type, expt_type, scale)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_corrects.double() / dataset_len

    return epoch_loss, epoch_acc

In [22]:
#for epoch in trange(epochs, leave=False):
loss, accuracy = train_loop(model,
                            expt_type,
                            train_dataloader,
                            optimizer,
                            device,
                            len(train_dataset),
                            loss_type,
                            scale)
if scheduler is not None:
    scheduler.step()

if loss >= best_loss:
    early_stop_counter += 1
else:
    best_model_wts = copy.deepcopy(model.state_dict())
    early_stop_counter = 0
    best_loss = loss

#if early_stop_counter == early_stop_limit:
 #   break

/home/dsail/hyolim/sismo-wsdm/dataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  tweets = torch.tensor(self.tweets[item], dtype=torch.float32)


In [23]:
def eval_loop(model, expt_type, dataloader, device, dataset_len, loss_type, scale=1):
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    fin_targets = []
    fin_outputs = []

    fin_conf = []

    for bi, inputs in enumerate(dataloader):
        labels, tweet_features, lens = inputs

        labels = labels#.to(device)
        tweet_features = tweet_features#.to(device)

        with torch.no_grad():
            output = model(tweet_features, lens, labels)

        _, preds = torch.max(output, 1)

        loss = loss_function(output, labels, loss_type, expt_type, scale)

        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

        fin_conf.append(output.cpu().detach().numpy())

        fin_targets.append(labels.cpu().detach().numpy())
        fin_outputs.append(preds.cpu().detach().numpy())

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = running_corrects.double() / dataset_len

    return epoch_loss, epoch_accuracy, np.hstack(fin_outputs), np.hstack(fin_targets), fin_conf

In [24]:
tc = time.time()
model.load_state_dict(best_model_wts)
_, _, y_pred, y_true, conf = eval_loop(model,
                                       expt_type,
                                       test_dataloader,
                                       device,
                                       len(test_dataset),
                                       loss_type,
                                       scale)

In [25]:
m = gr_metrics(y_pred, y_true)
m

(0.7142857142857143, 0.21739130434782608, 0.3333333333333333, 0.51)

In [26]:
classwise_FScores = class_FScore(y_pred, y_true, expt_type)
classwise_FScores

[0.7777777725925925, 0.46153845763313617, 0.0, 0.0, 0.0]

In [27]:
if 'Precision' in metrics_dict:
    metrics_dict['Precision'].append(m[0])
    metrics_dict['Recall'].append(m[1])
    metrics_dict['FScore'].append(m[2])
else:
    metrics_dict['Precision'] = [m[0]]
    metrics_dict['Recall'] = [m[1]]
    metrics_dict['FScore'] = [m[2]]

In [28]:
df = pd.DataFrame(metrics_dict)
df

,Precision,Recall,FScore
0,0.714286,0.217391,0.333333


# 4. 뒤부터 풀어보기, model부터

In [29]:
model

RedditModel(
  (dropout): Dropout(p=0.3, inplace=False)
  (fc_1): Linear(in_features=1024, out_features=1024, bias=True)
  (fc_2): Linear(in_features=1024, out_features=5, bias=True)
  (historic_model): MyLSTM(
    (dropout): Dropout(p=0.3, inplace=False)
    (lstm1): LSTM(768, 512, bidirectional=True)
    (atten1): Attention()
  )
)

In [30]:
expt_type

5

In [31]:
class RedditModel(nn.Module):
    def __init__(self, op_units=5, embedding_dim=768, hidden_dim=128, lstm_layer=1, dropout=0.5):
        super(RedditModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(p=dropout)

        self.fc_1 = nn.Linear(hidden_dim*2, hidden_dim*2)
        self.fc_2 = nn.Linear(hidden_dim*2, op_units)

        self.historic_model = MyLSTM(self.embedding_dim, self.hidden_dim, lstm_layer, dropout)

    def get_pred(self, feat):
        feat = self.fc_1(self.dropout(feat))
        return self.fc_2(feat)

    def forward(self, tweets, lengths, labels):
        h, _ = self.historic_model(tweets, lengths)
        if h.dim() == 1:
            h = h.unsqueeze(0)
        e = self.get_pred(h)

        return e

In [32]:
op_units=5
embedding_dim=768
hidden_dim=128
lstm_layer=1
dropout=0.5

In [33]:
dropout = nn.Dropout(p=dropout)

In [34]:
fc_1 = nn.Linear(hidden_dim*2, hidden_dim*2)
fc_1

Linear(in_features=256, out_features=256, bias=True)

In [35]:
fc_2 = nn.Linear(hidden_dim*2, op_units)
fc_2

Linear(in_features=256, out_features=5, bias=True)

## 4.1 MyLSTM 풀기

In [36]:
class MyLSTM(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=128, lstm_layer=1, dropout=0.6):
        super(MyLSTM, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.embedding_dim = embedding_dim
        self.lstm1 = nn.LSTM(input_size=self.embedding_dim,
                             hidden_size=hidden_dim,
                             num_layers=lstm_layer,
                             bidirectional=True)
        self.atten1 = Attention(hidden_dim*2, batch_first=True)  # 2 is bidrectional

    def forward(self, x, x_len):
        x = self.dropout(x)

        x = nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        out, (h_n, c_n) = self.lstm1(x)
        x, lengths = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        x, _ = self.atten1(x, lengths)  # skip connect

        return x, _

In [37]:
embedding_dim=768
hidden_dim=128
lstm_layer=1
dropout=0.6

In [38]:
dropout = 0.6

In [39]:
nn.LSTM(input_size=embedding_dim,
                 hidden_size=hidden_dim,
                 num_layers=lstm_layer,
                 bidirectional=True)

LSTM(768, 128, bidirectional=True)

### 4.1.1 Attention 풀기

In [40]:
class Attention(nn.Module):
    def __init__(self, hidden_size, batch_first=False):
        super(Attention, self).__init__()

        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.att_weights = nn.Parameter(torch.Tensor(1, hidden_size), requires_grad=True)

        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.att_weights:
            nn.init.uniform_(weight, -stdv, stdv)

    def get_mask(self):
        pass

    def forward(self, inputs, lengths):
        if self.batch_first:
            batch_size, max_len = inputs.size()[:2]
        else:
            max_len, batch_size = inputs.size()[:2]

        # apply attention layer
        weights = torch.bmm(inputs,
                            self.att_weights  # (1, hidden_size)
                            .permute(1, 0)  # (hidden_size, 1)
                            .unsqueeze(0)  # (1, hidden_size, 1)
                            .repeat(batch_size, 1, 1)  # (batch_size, hidden_size, 1)
                            )

        attentions = torch.softmax(F.relu(weights.squeeze()), dim=-1)

        # create mask based on the sentence lengths
#         mask = torch.ones(attentions.size(), requires_grad=False).to('cpu')
#         for i, l in enumerate(lengths):  # skip the first sentence
#             if l < max_len:
#                 mask[i, l:] = 0

#         # apply mask and renormalize attention scores (weights)
#         masked = attentions * mask
#         _sums = masked.sum(-1).unsqueeze(-1)  # sums per row

#         attentions = masked.div(_sums)

        # if attentions.dim() == 1:
        #     attentions = attentions.unsqueeze(1)

        # apply attention weights
        weighted = torch.mul(inputs, attentions.unsqueeze(-1).expand_as(inputs))

        # get the final fixed vector representations of the sentences
        representations = weighted.sum(1).squeeze()

        return representations, attentions


In [41]:
hidden_size = hidden_dim*2
batch_first = True

att_weights = nn.Parameter(torch.Tensor(1, hidden_size), requires_grad=True)

stdv = 1.0 / np.sqrt(hidden_size)
for weight in att_weights:
    nn.init.uniform_(weight, -stdv, stdv)

def get_mask(self):
    pass

In [42]:
for bi, inputs in enumerate(train_dataloader):
        labels, tweet_features, lens = inputs

        labels = labels#.to(device)
        tweet_features = tweet_features#.to(device)

In [43]:
type(labels)

torch.Tensor

In [44]:
labels.size()

torch.Size([8])

In [45]:
tweet_features.size()

torch.Size([8, 54, 768])

In [46]:
a=[[8],[8, 28, 768],[8]]

In [47]:
a[:2]

[[8], [8, 28, 768]]

In [48]:
inputs

[tensor([2, 1, 2, 1, 2, 1, 2, 1]),
 tensor([[[-0.0553,  0.1175,  0.0335,  ..., -0.0765,  0.0150, -0.0386],
          [-0.0865,  0.0567, -0.0039,  ..., -0.0753, -0.0367, -0.0458],
          [-0.0556,  0.0892, -0.0098,  ..., -0.1102,  0.0048, -0.0400],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.1390,  0.0848,  0.0417,  ..., -0.0604, -0.0009, -0.0137],
          [-0.1313,  0.0552,  0.0709,  ..., -0.1226,  0.0031,  0.0345],
          [-0.0814,  0.0769,  0.0271,  ..., -0.0803, -0.0298, -0.0536],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.1092,  0.1412,  0.0492,  ..., -0.0746, -0.0007,  0.

In [49]:
atten1 = Attention(hidden_dim*2, batch_first=True)

In [50]:
historic_model = MyLSTM(embedding_dim, hidden_dim, lstm_layer, dropout)

In [51]:
historic_model

MyLSTM(
  (dropout): Dropout(p=0.6, inplace=False)
  (lstm1): LSTM(768, 128, bidirectional=True)
  (atten1): Attention()
)

# 코드 푸는 순서가 Redditmodel <- MyLSTM <- Attention

# 다시 처음부터
# 5. data 구조 보기

### dataset.py

### 데이터배치와 반복자 생성하기
https://tutorials.pytorch.kr/beginner/text_sentiment_ngrams_tutorial.html

1. collate_fn()
    * 데이터 로더에서 미니배치 생성시, 배치 샘플러가 뽑은 데이터셋들을 합치는 역할
    * 모델로 보내기 전, DataLoader에서 생성된 샘플 배치로 동작함
    * collate_fn의 입력은 DataLoader에 배치 크기가 있는 배치 데이터임
    * 미리 선언된 데이터 처리 파이프라인에 따라 처리

2. def pad_collate_reddit(batch): 

In [52]:
import torch
import torch.nn as nn


def pad_collate_reddit(batch): 
    target = [item[0] for item in batch]
    tweet = [item[1] for item in batch]

    lens = [len(x) for x in tweet]

    tweet = nn.utils.rnn.pad_sequence(tweet, batch_first=True, padding_value=0)

    target = torch.tensor(target)
    lens = torch.tensor(lens)

    return [target, tweet, lens]

### custom dataset(커스텀 데이터셋)

https://wikidocs.net/57165

* 데이터셋을 상속받아 직접 커스텀 데이터셋을 만드는 것으로
* torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스

#### 커스텀 데이터셋 기본 뼈대
* class CustomDataset(torch.utils.data.Dataset): 
  * def __init__(self):           # 데이터셋의 전처리를 해주는 부분

  * def __len__(self):            # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
                                # 데이터셋의 크기를 리턴할 len
  * def __getitem__(self, idx):  # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
                               # dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item

In [53]:
from torch.utils.data import Dataset

class RedditDataset(Dataset):
    def __init__(self, labels, tweets, days=200):
        super().__init__()
        self.labels = labels
        self.tweets = tweets # 트위터의 게시물
        self.days = days

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        labels = torch.tensor(self.labels[item], dtype=torch.long)
        if self.days > len(self.tweets[item]):
            tweets = torch.tensor(self.tweets[item], dtype=torch.float32)
        else:
            tweets = torch.tensor(self.tweets[item][:self.days], dtype=torch.float32)
        
        return [labels, tweets]

## RedditDataset 코드 풀어보기

### def __init__

In [54]:
labels=df_train.label.values 
tweets=df_train.enc.values
days=200

### def __getitem__

In [55]:
import random
item=random.randint(1,400) # 1-400 사이의 랜덤 정수 가져오기
item

318

In [56]:
labels = torch.tensor(labels[item], dtype=torch.long)
labels

tensor(4)

In [57]:
if days > len(tweets[item]):
    tweets = torch.tensor(tweets[item], dtype=torch.float32)
else:
    tweets = torch.tensor(tweets[item][:days], dtype=torch.float32)

[labels, tweets]
len(tweets)

15

#### if문을 해주는 이유
* 해당 데이터의 경우 500명의 reddit posts이다.

* enc는 각 게시물에 대해서 768차원으로 인코딩한 리스트 -> enc를 tweet이라고 정의
* 게시물의 길이가 200보다 긴 경우, 200까지만 나오도록 조정해서 반환한다

In [58]:
len(max(df_train.Post)) # 사용자 중 가장 많은 게시물을 쓴 사람은 24개.

24

In [59]:
print(len(df_train.Post[3])) # 4 | 3번 사용자의 경우, 4개의 게시물이 있다
len(df_train.enc[3][0]) # 768
len(df_train.enc[3][1]) # 768
len(df_train.enc[3][2]) # 768
len(df_train.enc[3][3]) # 768
# len(df.enc[3][4]) # 3번 사용자는 게시물이 4개까지 있기에 0,1,2,3 -> 4는 존재하지 않는다
len(df_train.enc[3]) # 4

4


4

In [60]:
len(tweets[3]) # 768, else문 진행
# tweets[3]

768

In [61]:
len(torch.tensor(tweets[3][:days], dtype=torch.float32)) # 200
torch.tensor(tweets[3][:days], dtype=torch.float32)

/tmp/ipykernel_2773432/3627452317.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  len(torch.tensor(tweets[3][:days], dtype=torch.float32)) # 200
/tmp/ipykernel_2773432/3627452317.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(tweets[3][:days], dtype=torch.float32)


tensor([-4.3798e-02,  3.6652e-02, -1.2237e-02, -8.9592e-02,  1.1714e-01,
        -5.9329e-02, -2.4218e-02,  1.3143e-02,  4.5902e-02, -9.9838e-02,
        -4.4578e-02,  5.1750e-02,  1.9876e-02, -2.5739e-02,  4.7524e-02,
        -2.2292e-02, -6.2777e-02,  2.0296e-02,  1.6537e-02, -4.5113e-02,
        -6.7061e-02,  4.2000e-02,  1.4737e-03,  1.1524e-01,  1.1123e-02,
         3.3623e-02,  1.0772e-01,  1.4821e-01, -5.2686e-02, -4.2978e-03,
        -5.9954e-02, -6.1087e-02,  9.4013e-02, -9.2577e-03, -9.9320e-03,
         7.5362e-02,  8.9905e-02, -3.8388e-02, -8.4326e-02, -4.7914e-03,
         3.3067e-02,  2.7983e-02,  2.2044e-02, -3.6369e-02,  5.1264e-02,
        -1.0881e-02, -1.3514e-02, -3.3244e-04, -1.1853e-02,  4.1809e-03,
         3.7452e-02,  3.2590e-02, -4.5625e-02,  4.6694e-02, -9.5488e-02,
         8.0763e-02,  2.6517e-02,  3.1645e-02,  6.2567e-02, -4.7160e-02,
        -2.7078e-02, -8.1448e-02, -8.3191e-02, -4.0581e-02,  6.5065e-02,
        -1.4841e-02, -3.2725e-02, -4.6252e-02, -9.9

In [62]:
tweets[3][0]

tensor(-0.0438)

In [63]:
from torch.utils.data import DataLoader, Dataset 

train_dataset = RedditDataset(
            df_train.label.values, # labels에 해당
            df_train.enc.values    # posts에 해당
            ) 
train_dataset

In [64]:
train_dataloader = DataLoader(
            train_dataset, batch_size=batch_size, collate_fn=pad_collate_reddit, shuffle=True) 

test_dataset = RedditDataset(
            df_test.label.values, df_test.enc.values)

test_dataloader = DataLoader(
            test_dataset, batch_size=batch_size, collate_fn=pad_collate_reddit)

# 6. model 풀기

## 6.1 Input 확인

In [65]:
second_train_data=next(iter(train_dataloader))
inputs=second_train_data[1]

In [66]:
inputs

tensor([[[-8.6060e-02,  6.0873e-02,  3.8409e-02,  ..., -1.1287e-01,
          -2.7783e-02,  2.3558e-02],
         [-7.9226e-02,  7.7066e-02,  9.7964e-03,  ..., -1.1475e-01,
          -3.7525e-02, -1.2379e-03],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-1.1699e-01,  4.7306e-02,  6.1936e-02,  ..., -6.7025e-02,
          -2.2637e-03,  3.5783e-02],
         [-7.1922e-02,  8.4512e-02,  3.7833e-02,  ..., -1.5439e-02,
          -4.2933e-02,  9.0660e-02],
         [-9.8161e-02,  9.3266e-02, -1.3176e-02,  ..., -6.3279e-02,
          -5.7189e-02, -9.3285e-02],
         ...,
         [-1.0713e-01,  4

In [67]:
type(inputs)

torch.Tensor

## 6.2 class Attention

In [68]:
class Attention(nn.Module):
    def __init__(self, hidden_size, batch_first=False):
        super(Attention, self).__init__()

        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.att_weights = nn.Parameter(torch.Tensor(1, hidden_size), requires_grad=True)

        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.att_weights:
            nn.init.uniform_(weight, -stdv, stdv)

    def get_mask(self):
        pass

    def forward(self, inputs, lengths):
        if self.batch_first:
            batch_size, max_len = inputs.size()[:2]
        else:
            max_len, batch_size = inputs.size()[:2]

        # apply attention layer
        weights = torch.bmm(inputs,
                            self.att_weights  # (1, hidden_size)
                            .permute(1, 0)  # (hidden_size, 1)
                            .unsqueeze(0)  # (1, hidden_size, 1)
                            .repeat(batch_size, 1, 1)  # (batch_size, hidden_size, 1)
                            )

        attentions = torch.softmax(F.relu(weights.squeeze()), dim=-1)

        # create mask based on the sentence lengths
        mask = torch.ones(attentions.size(), requires_grad=True).to(device)# .cuda()
        for i, l in enumerate(lengths):  # skip the first sentence
            if l < max_len:
                mask[i, l:] = 0

        # apply mask and renormalize attention scores (weights)
        masked = attentions * mask
        _sums = masked.sum(-1).unsqueeze(-1)  # sums per row

        attentions = masked.div(_sums)

        # if attentions.dim() == 1:
        #     attentions = attentions.unsqueeze(1)

        # apply attention weights
        weighted = torch.mul(inputs, attentions.unsqueeze(-1).expand_as(inputs))

        # get the final fixed vector representations of the sentences
        representations = weighted.sum(1).squeeze()

        return representations, attentions

## Class Attention 코드 풀어보기

In [69]:
import numpy as np

hidden_size = 768 # hidden_size를 정의해준 부분이 없음 ??
batch_first=False # class마다 True, False가 다르므로 각각 명시해주어야 함
att_weights = nn.Parameter(torch.Tensor(1, hidden_size), requires_grad=True)
stdv = 1.0 / np.sqrt(hidden_size)

for weight in att_weights:
            nn.init.uniform_(weight, -stdv, stdv)
att_weights.size()

torch.Size([1, 768])

In [70]:
def get_mask(self):
    pass

In [71]:
second_train_data=next(iter(train_dataloader))
inputs=second_train_data[1]

In [72]:
batch_size, max_len = inputs.size()[:2] # [:2] 가 무슨기능????????//

In [73]:
max_len

130

In [74]:
batch_size

8

In [75]:
inputs.size()

torch.Size([8, 130, 768])

In [76]:
weights = torch.bmm(inputs,
                    att_weights  # (1, hidden_size)
                    .permute(1, 0)  # (hidden_size, 1)
                    .unsqueeze(0)  # (1, hidden_size, 1)
                    .repeat(batch_size, 1, 1)  # (batch_size, hidden_size, 1)
                    )
weights.size()

torch.Size([8, 130, 1])

In [77]:
weights.squeeze().size()

torch.Size([8, 130])

In [78]:
attentions = torch.softmax(F.relu(weights.squeeze()), dim=-1)
attentions.size()

torch.Size([8, 130])

In [79]:
weighted = torch.mul(inputs, attentions.unsqueeze(-1).expand_as(inputs))
weighted.size()

torch.Size([8, 130, 768])

In [80]:
representations = weighted.sum(1).squeeze()
representations.size()

torch.Size([8, 768])

## 6.3 MyLSTM 코드 풀어보기
* Redditmodel에서 MyLSTM(self.embedding_dim, self.hidden_dim, lstm_layer, dropout)으로 쓰임
* historic_model(tweets, lengths)

In [81]:
class MyLSTM(nn.Module):
    def __init__(self, embedding_dim=768, hidden_dim=128, lstm_layer=1, dropout=0.6):
        super(MyLSTM, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.embedding_dim = embedding_dim
        self.lstm1 = nn.LSTM(input_size=self.embedding_dim,
                             hidden_size=hidden_dim,
                             num_layers=lstm_layer,
                             bidirectional=True)
        self.atten1 = Attention(hidden_dim*2, batch_first=True)  # 2 is bidrectional

    def forward(self, x, x_len):
        x = self.dropout(x)

        x = nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        out, (h_n, c_n) = self.lstm1(x)
        x, lengths = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        x, _ = self.atten1(x, lengths)  # skip connect

        return x, _

In [82]:
embedding_dim=768
hidden_dim=128
lstm_layer=1
dropout=0.6

In [83]:
dropout = nn.Dropout(p=0.6)

In [84]:
lstm1 = nn.LSTM(input_size=embedding_dim,
                 hidden_size=hidden_dim,
                 num_layers=lstm_layer,
                 bidirectional=True)
lstm1

LSTM(768, 128, bidirectional=True)

In [85]:
atten1 = Attention(hidden_dim*2, batch_first=True)
atten1

Attention()

forward의 x는 tweets임

In [86]:
x=second_train_data[1]
x_len=second_train_data[2]

In [87]:
x=dropout(x)
x.size()

torch.Size([8, 130, 768])

In [88]:
x = nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
x

PackedSequence(data=tensor([[-0.1392,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.1102],
        [-0.0000,  0.0000, -0.0146,  ..., -0.3217, -0.0000, -0.0000],
        [-0.1089,  0.2633,  0.0000,  ..., -0.0000,  0.0188, -0.0000],
        ...,
        [-0.0000,  0.2827,  0.0000,  ..., -0.0000, -0.0000, -0.0844],
        [-0.0000,  0.0000,  0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [-0.0640,  0.0000,  0.0599,  ..., -0.0000, -0.0000, -0.0000]]), batch_sizes=tensor([8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 5, 5, 5, 3, 3,
        3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), sorted_indices=tensor([7, 5, 0, 4, 6, 3, 1, 2]), unsorted_indices=tensor([2, 6, 7, 5, 3, 1, 4

In [89]:
out, (h_n, c_n) = lstm1(x)

In [90]:
out

PackedSequence(data=tensor([[-0.0238,  0.0403, -0.0090,  ..., -0.0942,  0.1965, -0.0975],
        [-0.0327, -0.0106,  0.0120,  ..., -0.0731,  0.2535, -0.1225],
        [-0.0123,  0.0099,  0.0963,  ..., -0.1105,  0.2100, -0.3535],
        ...,
        [-0.0514,  0.0154,  0.1713,  ..., -0.1770,  0.1341, -0.3069],
        [-0.0328,  0.0184,  0.1802,  ..., -0.1106,  0.0651, -0.2461],
        [-0.0163,  0.0553,  0.2955,  ..., -0.0127, -0.0314, -0.0466]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 5, 5, 5, 3, 3,
        3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), sorted_indices=tensor([7, 5, 0, 4, 6, 3, 1, 2]), unsorted_indi

In [91]:
(h_n, c_n)

(tensor([[[ 0.0459,  0.0664,  0.0158,  ...,  0.0294,  0.1332,  0.0370],
          [-0.0163,  0.0108,  0.1185,  ..., -0.0130,  0.2359,  0.0730],
          [-0.0201,  0.0121,  0.1176,  ..., -0.0149,  0.1274, -0.0395],
          ...,
          [-0.0083,  0.0177,  0.0898,  ..., -0.0201,  0.4023,  0.0788],
          [ 0.0283,  0.0650,  0.2173,  ..., -0.0226,  0.2053,  0.1024],
          [-0.0163,  0.0553,  0.2955,  ..., -0.1477,  0.2371,  0.1710]],
 
         [[-0.1586,  0.3025, -0.0340,  ..., -0.1105,  0.2100, -0.3535],
          [ 0.1394, -0.0116,  0.0401,  ..., -0.0131,  0.0197, -0.1109],
          [-0.0487,  0.1723,  0.0039,  ..., -0.0881,  0.0933, -0.2178],
          ...,
          [ 0.0953,  0.3639,  0.0100,  ..., -0.0731,  0.2535, -0.1225],
          [ 0.0357,  0.2682,  0.0586,  ..., -0.0024,  0.1811, -0.1722],
          [ 0.1699,  0.2039,  0.0097,  ..., -0.0942,  0.1965, -0.0975]]],
        grad_fn=<IndexSelectBackward0>),
 tensor([[[ 0.0925,  0.1232,  0.0307,  ...,  0.0625,  0.2995

In [92]:
x, lengths = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
x.size()

torch.Size([8, 130, 256])

## 6.4 RedditModel 코드 풀어보기

In [93]:
config['model_type'] == 'lstm+att'

True

In [94]:
class RedditModel(nn.Module):
    def __init__(self, op_units=5, embedding_dim=768, hidden_dim=128, lstm_layer=1, dropout=0.5):
        super(RedditModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(p=dropout)

        self.fc_1 = nn.Linear(hidden_dim*2, hidden_dim*2)
        self.fc_2 = nn.Linear(hidden_dim*2, op_units)

        self.historic_model = MyLSTM(self.embedding_dim, self.hidden_dim, lstm_layer, dropout)

    def get_pred(self, feat):
        feat = self.fc_1(self.dropout(feat))
        return self.fc_2(feat)

    def forward(self, tweets, lengths, labels):
        h, _ = self.historic_model(tweets, lengths)
        if h.dim() == 1:
            h = h.unsqueeze(0)
        e = self.get_pred(h)

        return e

In [95]:
op_units=5
embedding_dim=768
hidden_dim=128
lstm_layer=1
dropout=0.5

In [96]:
dropout = nn.Dropout(p=0.5)
fc_1 = nn.Linear(hidden_dim*2, hidden_dim*2)
fc_2 = nn.Linear(hidden_dim*2, op_units)

In [97]:
historic_model = MyLSTM(embedding_dim, hidden_dim, lstm_layer, 0.5)

forward 진행 전 인풋 확인

In [98]:
second_train_data

[tensor([4, 1, 2, 2, 3, 2, 1, 1]),
 tensor([[[-0.0436,  0.1053,  0.0006,  ..., -0.0890,  0.0075, -0.1056],
          [-0.0679,  0.0835,  0.0467,  ..., -0.0658, -0.0073,  0.0197],
          [-0.0689,  0.0453,  0.0218,  ..., -0.0969, -0.0332, -0.0208],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.0753,  0.0501, -0.0393,  ..., -0.1116, -0.0169,  0.0050],
          [-0.0898,  0.0147, -0.0285,  ..., -0.1240, -0.0172,  0.0275],
          [-0.0141,  0.0909,  0.0078,  ..., -0.1082,  0.0224, -0.0661],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.1121,  0.1177,  0.0401,  ..., -0.0817,  0.0015,  0.

In [99]:
lables=second_train_data[0]
tweets=second_train_data[1]
lengths=second_train_data[2]

In [100]:
h, _ = historic_model(tweets, lengths)

RuntimeError: a view of a leaf Variable that requires grad is being used in an in-place operation.

In [ ]:
def get_pred(self, feat):
    feat = self.fc_1(self.dropout(feat))
    return self.fc_2(feat)

In [ ]:
if h.dim() == 1:
    h = h.unsqueeze(0)
e = get_pred(h)
e